# Export to Excel tutorial

Unfortunately, it is not always straightforward to export results from Python to Excel, especially to an existing workbook.
In this notebook, we will look at a few examples.

## Preparation: Data import and preparation

This step is just to get some sample data, similar to the first assignment.

In [14]:
#Importing necessary packages
import pandas as pd

# Extracting data from the source file into a panda Dataframe
file = "Exercise_1_Data.xlsx"
# This file should be placed in the same directory as this notebook (on the JupyterHub server)

# We read the data and store it in a Dataframe called "data", using the read_excel() panda function
data = pd.read_excel(file, sheet_name = 'CarStockData', usecols='D:P', skiprows=4 )
# sheet_name, usecols and skiprows help us select the right data

# This is just to replace all the void values in "data" by 0, which makes things easier
data.fillna(0, inplace=True)

We randomly modify a bit the data, so that it looks a bit different from the original one. Of course, in the assignment, you need to use dynamic MFA methods to calculate all missing values.

In [15]:
data['Outflow'] = data['Time'] - data['Inflow'] 
data['Stock Change'] = data['Inflow'] - data['Outflow']
data['Total km driven'] =  data['Stock'] * data['Kilometrage per vehicle']

Create two Dataframes from data:
 - historic_data with values up to 2008
 - future_data with values for 2009 and after


## Export the whole dataframe to a new workbook

This is probably the easiest method: you dump all the data to a new workbook. Then, it is really easy to either copy-paste it manually in another workbook or using Excel formulas to update automatically. 

Warning: This method would not work with an existing workbook, as it would be erased without warning.

We use the pandas.DataFrame.to_excel function, but we might have to install the xlwt package first:

In [16]:
"""
The first time you try this, you might need to run this code first 
to install the xlwt package in the jupyter environment
"""

import sys
!{sys.executable} -m pip install xlwt

In [17]:
# This will export the data in a new file named Results.xlsx, in a sheet named Results. 
# Check the help of this function for the meaning of additional parameters

data.to_excel('Results.xls', sheet_name='Results')

You will get a new Excel file in your Jupyter directory, that you can download and modify locally on your computer.

Note: it is possible to do the same thing with the xlsxwriter package.

## Exports the results to an existing workbook

Neither the to_excel function in pandas nor the xlsxwriter packages enable export to an existing workbook without erasing it. This is because these functions can only write in a workbook, not read it.

Therefore, to write in an existing workbook, we need to install the openpyxl package:

In [18]:
import sys
!{sys.executable} -m pip install openpyxl

There are multiple ways to use this package. Notably, you can read and edit single cells, a range of cells, or you can iterate on them. You can also do some formatting etc., even if this would probably be easier directly in Excel. Do not hesitate to check the documentation for tutorials and examples.

Below is an example of a function found on <a href="https://stackoverflow.com/questions/36447758/writing-pandas-data-frame-to-existing-workbook">stackoverflow</a> that enable to export a Dataframe to an existing Excel workbook without erasing existing the other content. Note: it only works with .xlsx files, so to use it on the original data file of the exercise, you will need  to convert it from .xls to .xlsx before reuploading it to the Jupyter server.


In [19]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

And here is an example of how to use this function for the first assignment (note the use of the parameters to write in the right place in the worksheet):

In [20]:
# Works only for .xlsx files
append_df_to_excel('Exercise_1_Data.xlsx', data, sheet_name="CarStockData", startrow=4, startcol=3, index=False)